Initialise Project Root for imports

In [2]:
import sys
from pathlib import Path

# Add the project root to Python path
project_root = str(Path.cwd().parent)
sys.path.append(project_root)

In [17]:

from utils import read_csv

df_find_your_flow = read_csv("../assets/export-tickuserstate-20241230_161045.csv")
df_promotional_ideas = read_csv("../assets/promotional-ideas-tickuserstate-20250306_123551.csv")

df = df_promotional_ideas
df

,sessionID,sessionStateID,userID,artistID,sessionStateStatus,sessionStateDateCreated,sessionStateDateUpdated,tickStateID,tickStateStatus,tickStateDateCreated,tickStateDateUpdated,tickID,tickType,tickHandler,tickStateResult
0,35,95881,184314,10375,5,2025-03-06 12:27,2025-03-06 12:35,850891,5,2025-03-06 12:35,2025-03-06 12:35,470,1,NaN,"{\n ""result"": 1\n}"
1,35,95881,184314,10375,5,2025-03-06 12:27,2025-03-06 12:35,850890,5,2025-03-06 12:34,2025-03-06 12:34,469,10,llmRequest,NaN
2,35,95881,184314,10375,5,2025-03-06 12:27,2025-03-06 12:35,850888,5,2025-03-06 12:34,2025-03-06 12:34,462,2,NaN,"{\n ""response_rating"": ""no"",\n ""response..."
3,35,95881,184314,10375,5,2025-03-06 12:27,2025-03-06 12:35,850887,5,2025-03-06 12:34,2025-03-06 12:34,461,10,llmRequest,NaN
4,35,95881,184314,10375,5,2025-03-06 12:27,2025-03-06 12:35,850885,5,2025-03-06 12:34,2025-03-06 12:34,460,2,NaN,"{\n ""response_rating"": ""other"",\n ""respo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,35,41932,54,5284,8,2024-09-09 12:43,2024-12-05 17:53,353201,5,2024-09-09 12:43,2024-09-09 12:43,454,2,NaN,"{\n ""inputValue"": ""really""\n}"
3486,35,41930,54,5284,8,2024-09-09 12:40,2024-12-05 17:53,353190,5,2024-09-09 12:40,2024-09-09 12:40,457,10,llmRequest,NaN
3487,35,41930,54,5284,8,2024-09-09 12:40,2024-12-05 17:53,353189,5,2024-09-09 12:40,2024-09-09 12:40,456,2,NaN,"{\n ""inputValue"": ""your ex realising he lef..."
3488,35,41930,54,5284,8,2024-09-09 12:40,2024-12-05 17:53,353188,5,2024-09-09 12:40,2024-09-09 12:40,455,2,NaN,"{\n ""inputValue"": [\n ""upcoming""\n ..."


In [18]:
tickMap_find_your_flow ={
  38: "What can I call you?",
  39: "What is your role?",
  40: "Search on spotify",
  42: "What are your pronouns",
  72:"Let's get started",
  59: "Authentication",
  70: "Create artist"
}

tickMap_promotional_ideas = {
  454: "Input name",
  455: "Input state",
  456: "Input description",
  457: "LLM 1",
  458: "Result 1",
  466: "Save 1",
  459: "LLM 2",
  460: "Result 2",
  467: "Save 2",
  461: "LLM 3",
  462: "Result 3",
  468: "Save 3",
  469: "LLM 4",
  470: "Not Happy with ALL result",
}

 # Map of tickID to tickName
ticksMap=tickMap_promotional_ideas

In [21]:
import pandas as pd

# Generate a collection of tickID to number of occurrences
tick_id_occurrences = df["tickID"].value_counts() # total tick states
tick_df = pd.DataFrame(tick_id_occurrences).reset_index()
tick_df.columns = ["tickID", "count"]
tick_df["tickName"] = tick_df["tickID"].map(ticksMap)
# Filter out rows with NaN values in tickName col
tick_df = tick_df[tick_df["tickName"].notna()]
tick_df["percentage"] = round((tick_df["count"] / tick_df["count"].iloc[0]) * 100, 2)
tick_df["tickName"] = tick_df["tickID"].map(ticksMap)
tick_df["dropOffRateCum"] = round((tick_df["count"].iloc[0] - tick_df["count"]) / tick_df["count"].iloc[0] * 100, 2)
# Subtract the current percentage from the previous percentage
tick_df["dropOffRateDiff"] = tick_df["percentage"].diff() * -1
# Calculate percentage drop off based on dropOffRateDiff/percentage of previous tick
tick_df["dropOffRateRelative"] = round((tick_df["dropOffRateDiff"] / tick_df["percentage"].shift(1)) * 100, 2)

tick_df 

,tickID,count,tickName,percentage,dropOffRateCum,dropOffRateDiff,dropOffRateRelative
0,454,600,Input name,100.00,0.00,NaN,NaN
1,455,595,Input state,99.17,0.83,0.83,0.83
2,456,566,Input description,94.33,5.67,4.84,4.88
3,457,566,LLM 1,94.33,5.67,-0.00,-0.00
4,458,510,Result 1,85.00,15.00,9.33,9.89
5,466,487,Save 1,81.17,18.83,3.83,4.51
7,459,23,LLM 2,3.83,96.17,77.34,95.28
8,460,17,Result 2,2.83,97.17,1.00,26.11
9,461,11,LLM 3,1.83,98.17,1.00,35.34
10,462,9,Result 3,1.50,98.50,0.33,18.03
